# 필요한 모듈 설치

In [2]:
!pip install tqdm

# 필요한 모듈들 import

In [3]:
# 1. 시스템 및 환경 관련 라이브러리
import os  # 운영체제 관련 기능을 제공하는 모듈 (파일 경로 처리 등)
import random  # 무작위 수를 생성하거나 랜덤화할 때 사용하는 모듈
import tqdm  # 반복문 진행 상황을 시각적으로 보여주는 모듈 (진행 바 표시)

# 2. 데이터 분석 및 처리 라이브러리
import pandas as pd  # 데이터 처리 및 분석을 위한 라이브러리 (특히 DataFrame 사용)
import numpy as np  # 수학적 계산 및 배열 작업을 위한 라이브러리 (벡터화 연산)

# 3. 머신러닝 관련 라이브러리
from sklearn.model_selection import train_test_split  # 데이터를 훈련 세트와 테스트 세트로 나누는 기능
from sklearn.preprocessing import OneHotEncoder  # 범주형 데이터를 원-핫 인코딩하는 클래스
from sklearn.tree import DecisionTreeClassifier  # 결정 트리 분류 모델을 위한 클래스
from sklearn.metrics import roc_curve, auc  # ROC 곡선 계산 및 AUC 값 평가를 위한 함수

# 4. 딥러닝 및 GPU 연산 관련 라이브러리
import torch  # 딥러닝 모델 및 GPU 연산을 위한 주요 라이브러리 (PyTorch)

from datetime import datetime  # 날짜와 시간 처리 모듈

# 추가적인 sklearn 모듈
from sklearn.model_selection import train_test_split  # (중복된 임포트, 이미 위에서 임포트됨)
from sklearn.preprocessing import OneHotEncoder  # (중복된 임포트, 이미 위에서 임포트됨)

# 현재 날짜와 시간 정보를 가져옵니다.
today = datetime.today().strftime('%m%d')  # 오늘 날짜 (예: 0207)

# seed 값을 고정하는 함수
def reset_seeds(seed=42):
    random.seed(seed)  # random 모듈의 시드 고정
    os.environ['PYTHONHASHSEED'] = str(seed)  # 파이썬 해시 시드 고정
    np.random.seed(seed)  # numpy의 시드 고정
    torch.manual_seed(seed)  # PyTorch CPU 연산의 시드 고정
    torch.cuda.manual_seed(seed)  # PyTorch GPU 연산의 시드 고정
    torch.backends.cudnn.deterministic = True  # CUDA 라이브러리에서 결정론적 예측 설정

# 제출 파일 이름을 생성하는 함수
def get_submission_filename(base_path, base_filename):
    counter = 1  # 파일 이름 중복 방지를 위한 카운터 초기화
    submission_path = os.path.join(base_path, f"{base_filename}_{today}_{counter}.csv")

    # 해당 경로에 파일이 존재하는지 확인하고, 존재하면 counter를 증가시켜 반복
    while os.path.exists(submission_path):
        counter += 1
        submission_path = os.path.join(base_path, f"{base_filename}_{today}_{counter}.csv")

    return submission_path


# Connect to googld drive

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Global Variables

In [5]:
import easydict
args = easydict.EasyDict()

# path info
args.default_path = "/content/drive/MyDrive/Colab Notebooks/AI 기초_KDT/강의/kaggle/data/"
args.train_csv = args.default_path + "train.csv"
args.test_csv = args.default_path + "test.csv"
args.default_submission = args.default_path + "submission.csv"
args.submit_submission = "/content/drive/MyDrive/Colab Notebooks/AI 기초_KDT/강의/kaggle/submission_csv/"

# 제출용
args.submission_csv = get_submission_filename(args.submit_submission, "submission")
args.submission_csv

'/content/drive/MyDrive/Colab Notebooks/AI 기초_KDT/강의/kaggle/submission_csv/submission_0207_1.csv'

# Load Titanic Dataset
- Surived:0=사망, 1=생존
- Pclass: 1=1등석, 2=2등석, 3=3등석
- gender:male=남성, female=여성
- Age: 나이
- SibSp: 타이타닉 호에 동승한 자매/배우자의 수
- Parch: 타이타닉 호에 동승한 부모/자식의 수
- Ticket: 티켓 번호
- Fare: 승객 요금
- Cabin: 방 호수
- Embarked: 탑승지; C=셰르부르, Q=퀴즈타운, S=사우샘프턴

In [6]:
ori_train = pd.read_csv(args.train_csv)
ori_test = pd.read_csv(args.test_csv)
default_submission = pd.read_csv(args.default_submission)

ori_train.shape, ori_test.shape, default_submission.shape

((916, 12), (393, 11), (393, 2))

In [7]:
default_submission.head()

,passengerid,survived
0,916,0.5
1,917,0.5
2,918,0.5
3,919,0.5
4,920,0.5


In [8]:
ori_train.head()

,passengerid,survived,pclass,name,gender,age,sibsp,parch,ticket,fare,cabin,embarked
0,0,0,2,"Wheeler, Mr. Edwin Frederick""""",male,NaN,0,0,SC/PARIS 2159,12.8750,NaN,S
1,1,0,3,"Henry, Miss. Delia",female,NaN,0,0,382649,7.7500,NaN,Q
2,2,1,1,"Hays, Mrs. Charles Melville (Clara Jennings Gr...",female,52.0,1,1,12749,93.5000,B69,S
3,3,1,3,"Andersson, Mr. August Edvard (""Wennerstrom"")",male,27.0,0,0,350043,7.7958,NaN,S
4,4,0,2,"Hold, Mr. Stephen",male,44.0,1,0,26707,26.0000,NaN,S


In [9]:
ori_train.columns

Index(['passengerid', 'survived', 'pclass', 'name', 'gender', 'age', 'sibsp',
       'parch', 'ticket', 'fare', 'cabin', 'embarked'],
      dtype='object')

In [10]:
ori_test.columns

Index(['passengerid', 'pclass', 'name', 'gender', 'age', 'sibsp', 'parch',
       'ticket', 'fare', 'cabin', 'embarked'],
      dtype='object')

- test 데이터셋에는 survived가 없음을 알 수 있음
- train 데이터셋에서 훈련을 위한 데이터프레임과 검증을 위한 데이터프레임을 나눠야 함.

# Train & Test Split

In [11]:
y = ori_train['survived']
X = ori_train.drop(['survived'], axis=1)

In [12]:
reset_seeds()
X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.2, stratify=ori_train['survived'])

X_tr.shape, X_te.shape, y_tr.shape, y_te.shape

((732, 11), (184, 11), (732,), (184,))

# Data Preprocessing

In [13]:
train = X_tr.copy()
test = X_te.copy()
ori_te = ori_test.copy()

train.shape, test.shape, ori_te.shape

((732, 11), (184, 11), (393, 11))

## EDA

In [14]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 732 entries, 914 to 636
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   passengerid  732 non-null    int64  
 1   pclass       732 non-null    int64  
 2   name         732 non-null    object 
 3   gender       732 non-null    object 
 4   age          597 non-null    float64
 5   sibsp        732 non-null    int64  
 6   parch        732 non-null    int64  
 7   ticket       732 non-null    object 
 8   fare         732 non-null    float64
 9   cabin        152 non-null    object 
 10  embarked     732 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 68.6+ KB


## Data Cleaning

In [15]:
train.isnull().sum().sort_values(ascending=False)

,0
cabin,580
age,135
passengerid,0
pclass,0
name,0
gender,0
sibsp,0
parch,0
ticket,0
fare,0


In [16]:
test.isnull().sum().sort_values(ascending=False)

,0
cabin,138
age,45
embarked,1
passengerid,0
pclass,0
name,0
gender,0
sibsp,0
parch,0
ticket,0


In [17]:
ori_te.isnull().sum().sort_values(ascending=False)

,0
cabin,296
age,83
fare,1
embarked,1
passengerid,0
pclass,0
name,0
gender,0
sibsp,0
parch,0


In [18]:
age_median = train['age'].median()
fare_median = train['fare'].median()

# embarked와 cabin에 대해 mode()를 사용하는 이유는 두 열이 범주형 데이터이기 때문
embarked_mode = train['embarked'].mode().values[0]
cabin_mode = train['cabin'].mode().values[0]

age_median, fare_median, embarked_mode, cabin_mode

(28.0, 14.4583, 'S', 'F2')

In [19]:
train['age'].fillna(age_median, inplace=True)
test['age'].fillna(age_median, inplace=True)
ori_te['age'].fillna(age_median, inplace=True)

train['fare'].fillna(fare_median, inplace=True)
test['fare'].fillna(fare_median, inplace=True)
ori_te['fare'].fillna(fare_median, inplace=True)

train['embarked'].fillna(embarked_mode, inplace=True)
test['embarked'].fillna(embarked_mode, inplace=True)
ori_te['embarked'].fillna(embarked_mode, inplace=True)

train['cabin'].fillna(cabin_mode, inplace=True)
test['cabin'].fillna(cabin_mode, inplace=True)
ori_te['cabin'].fillna(cabin_mode, inplace=True)

train.isnull().sum().sum(), test.isnull().sum().sum(), ori_te.isnull().sum().sum()

<ipython-input-19-a58a9255a4f5>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train['age'].fillna(age_median, inplace=True)
<ipython-input-19-a58a9255a4f5>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using '

(0, 0, 0)

In [20]:
print(f'before: {train.shape} / {test.shape}')
drop_cols = ['name', 'ticket', 'cabin']

train.drop(drop_cols, axis=1, inplace=True) # 모델이 학습하는데 사용하는 데이터
test.drop(drop_cols, axis=1, inplace=True) # 모델의 학습을 평가(잘했는지?? 못했는지??)하기 위한 데이터
ori_te.drop(drop_cols, axis=1, inplace=True) # 학습이 잘된 모델을 이용해서 내가 알고 싶은(ori_te) 데이터를 예측하게 하는 것

print(f'after: {train.shape} / {test.shape}')
train.info()

before: (732, 11) / (184, 11)
after: (732, 8) / (184, 8)
<class 'pandas.core.frame.DataFrame'>
Index: 732 entries, 914 to 636
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   passengerid  732 non-null    int64  
 1   pclass       732 non-null    int64  
 2   gender       732 non-null    object 
 3   age          732 non-null    float64
 4   sibsp        732 non-null    int64  
 5   parch        732 non-null    int64  
 6   fare         732 non-null    float64
 7   embarked     732 non-null    object 
dtypes: float64(2), int64(4), object(2)
memory usage: 51.5+ KB


## Data Encoding

In [21]:
enc_cols = ['gender', 'embarked']
normal_cols = list(set(train.columns) - set(enc_cols))
normal_cols

['pclass', 'parch', 'age', 'passengerid', 'fare', 'sibsp']

In [22]:
reset_seeds()
print(f'before: {train.shape} / {test.shape}')

enc = OneHotEncoder()
# train
tmp_tr = pd.DataFrame(
    enc.fit_transform(train[enc_cols]).toarray(),
    columns = enc.get_feature_names_out()
)
enc_tr = pd.concat(
    [train[normal_cols].reset_index(drop=True), tmp_tr.reset_index(drop=True)]
    , axis=1
)
# test
tmp_te = pd.DataFrame(
    enc.transform(test[enc_cols]).toarray(),
    columns = enc.get_feature_names_out()
)
enc_te = pd.concat(
    [test[normal_cols].reset_index(drop=True), tmp_te.reset_index(drop=True)]
    , axis=1
)
# ori_test
tmp_te = pd.DataFrame(
    enc.transform(ori_te[enc_cols]).toarray(),
    columns = enc.get_feature_names_out()
)
enc_ori_te = pd.concat(
    [ori_te[normal_cols].reset_index(drop=True), tmp_te.reset_index(drop=True)]
    , axis=1
)

print(f'after: {enc_tr.shape} / {enc_te.shape}')
enc_tr.head()

before: (732, 8) / (184, 8)
after: (732, 11) / (184, 11)


,pclass,parch,age,passengerid,fare,sibsp,gender_female,gender_male,embarked_C,embarked_Q,embarked_S
0,3,0,28.0,914,7.7333,0,1.0,0.0,0.0,1.0,0.0
1,3,2,6.0,805,31.2750,4,1.0,0.0,0.0,0.0,1.0
2,3,1,1.0,255,12.1833,1,1.0,0.0,0.0,0.0,1.0
3,3,0,70.5,769,7.7500,0,0.0,1.0,0.0,1.0,0.0
4,3,0,35.0,527,7.1250,0,0.0,1.0,0.0,0.0,1.0


# Training

In [23]:
enc_tr.isnull().sum().sum(), enc_te.isnull().sum().sum(), enc_ori_te.isnull().sum().sum()

(0, 0, 0)

In [24]:
# row는 다르더라도 column은 같아야 함
enc_tr.shape, enc_te.shape, enc_ori_te.shape

((732, 11), (184, 11), (393, 11))

In [25]:
reset_seeds()

modelV0 = DecisionTreeClassifier()

print(f'{enc_tr.shape} / {y_tr.shape}')
modelV0.fit(enc_tr, y_tr)

(732, 11) / (732,)


DecisionTreeClassifier()

# Evaluation

In [26]:
reset_seeds()

# Train data -> 학습O 데이터
score_tr = modelV0.score(enc_tr, y_tr)

# Test data -> 학습X 데이터
score_te = modelV0.score(enc_te, y_te)

score_tr, score_te

(1.0, 0.8532608695652174)

### AUC 점수

In [27]:
y_pred = modelV0.predict_proba(enc_te)[:,1]
fpr, tpr, thresholds = roc_curve(y_te, y_pred)

auc_te = auc(fpr, tpr)
print(f'model: {auc_te}')

model: 0.8540100250626567


# Submission

In [28]:
ori_te_pred = modelV0.predict_proba(enc_ori_te)[:,1]
ori_te_pred.shape

(393,)

In [29]:
default_submission.head()

,passengerid,survived
0,916,0.5
1,917,0.5
2,918,0.5
3,919,0.5
4,920,0.5


In [30]:
default_submission['survived'] = ori_te_pred
print(f"{default_submission.isnull().sum().sum()}")
default_submission.head()

0


,passengerid,survived
0,916,1.0
1,917,1.0
2,918,1.0
3,919,0.0
4,920,1.0


In [31]:
default_submission.to_csv(args.submission_csv, header=True, index=False)